# Word-level embedding and query search engine

Broadly speaking, your task will be to:
- Train a fasttext model on War and Peace by Leo Tolstoy
- Finetune (continue training) obtained embeddings using quora questions in order to account for new tokens and ever-changing semantic meaning
- Embed existing quora questions and store them for later use
- Implement a way to search the closest match among quora questions for an unseen text

# Training initial embeddings

Your first task will be to train embeddings on War and Peace by Leo Tolstoy. using a word2vec extention called fasttext.

There are certain differences with a vanilla word2vec, which you can read about [here](https://github.com/facebookresearch/fastText?tab=readme-ov-file), for example, but for all intents and purposes it is easier to treat fasttext as a word2vec on steroids.

The most important difference, arguably, is that the model vocabulary is constructed not only using word-level tokens, but also their n-grams e.g. for a word "peace" the resulting tokens using 2-grams would look like (peace, pe, ea, ac, ce).

This allows the model not only to handle out-of-vocabulary words by combining the corresponding subword tokens, but also to lift the need for word normalization (stemming, lemmatization) in some cases, preserving semantics in morphology-rich languages.

In [2]:
!pip install gensim
!pip install unidecode
!pip install faiss-cpu

  Using cached Unidecode-1.4.0-py3-none-any.whl.metadata (13 kB)
Using cached Unidecode-1.4.0-py3-none-any.whl (235 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 31.4/31.4 MB 34.2 MB/s eta 0:00:00


In [3]:

import re
from typing import Union, List, Tuple, Callable
import string
import nltk
import faiss
import numpy as np
from unidecode import unidecode
from gensim.models import FastText
from gensim.models.word2vec import PathLineSentences

nltk.download('punkt_tab')

# Set seed for reproducibility
# Note that in order to do the computations trully determenistic
# it is required to run the code using a sigle thread
seed = 42

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


Load war and peace text file from Gutenberg project repository.

In [4]:
!wget "https://www.gutenberg.org/files/2600/2600-0.txt" -O war_and_peace_raw.txt

--2025-10-09 19:42:03--  https://www.gutenberg.org/files/2600/2600-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3359405 (3.2M) [text/plain]
Saving to: ‘war_and_peace_raw.txt’

war_and_peace_raw.t 100%[===================>]   3.20M  19.7MB/s    in 0.2s    

2025-10-09 19:42:04 (19.7 MB/s) - ‘war_and_peace_raw.txt’ saved [3359405/3359405]



In [5]:
wap_raw_file_path = 'war_and_peace_raw.txt'
wap_cleaned_file_path = 'war_and_peace_cleaned.txt'

Note that the text contains information redundant for the task, so we need to preprocess the file first.

In [6]:
!head war_and_peace_raw.txt

The Project Gutenberg eBook of War and Peace, by Leo Tolstoy

This eBook is for the use of anyone anywhere in the United States and
most other parts of the world at no cost and with almost no restrictions
whatsoever. You may copy it, give it away or re-use it under the terms
of the Project Gutenberg License included with this eBook or online at
www.gutenberg.org. If you are not located in the United States, you
will have to check the laws of the country where you are located before
using this eBook.



In [7]:
# Load the text from the file
with open(wap_raw_file_path, 'r', encoding='utf-8') as file:
    text = file.read()

# Remove the header and footer
start_marker = "*** START OF THE PROJECT GUTENBERG EBOOK WAR AND PEACE ***"
end_marker = "*** END OF THE PROJECT GUTENBERG EBOOK WAR AND PEACE ***"
text = text.split(start_marker, 1)[-1]  # Remove everything before the start marker
text = text.split(end_marker, 1)[0]     # Remove everything after the end marker

# Split the text into lines and filter out unwanted ones
lines = text.split("\n")
filtered_lines = []
for line in lines:
    line = line.strip()
    # Remove lines that are empty or contain chapter headers or non-content information
    if not line or line.startswith("CHAPTER") or line.isupper():
        continue
    filtered_lines.append(line)

# Join the lines back into a single string
cleaned_text = "\n".join(filtered_lines[2:])

The text contains some non-ascii characters for historical reasons. One way to deal with it is to cast them to the closest ascii equivalent.

Note that it is not always required and in some cases may affect the model performance in a bad way.

*In short, if you feel that these characters are needed, leave them be.*

In [8]:
# Normalize to ASCII
cleaned_text_ascii = unidecode(cleaned_text)

Another important thing is to know your data and how the model trains its weights.

In our case, word2vec (fasttext) processes the text line by line, traversing the line with a defined window to represent the context. However, the Gutenberg War and Peace text is split into lines in a somewhat arbitrary way, so that the words on a particular line may not necesserily belong to the same sentence and thus the context window will erroneously treat them as contextually-dependent.

One way to fix it is to tokenize the text into sentences first. Luckily for us, the whole War and Peace text fits into the RAM.

In [9]:
# Split into sentences
sentences = nltk.sent_tokenize(cleaned_text_ascii)

sentences[:10]

['"Well, Prince, so Genoa and Lucca are now just family estates of the\nBuonapartes.',
 "But I warn you, if you don't tell me that this means war,\nif you still try to defend the infamies and horrors perpetrated by that\nAntichrist--I really believe he is Antichrist--I will have nothing\nmore to do with you and you are no longer my friend, no longer my\n'faithful slave,' as you call yourself!",
 'But how do you do?',
 'I see I\nhave frightened you--sit down and tell me all the news."',
 'It was in July, 1805, and the speaker was the well-known Anna Pavlovna\nScherer, maid of honor and favorite of the Empress Marya Fedorovna.',
 'With these words she greeted Prince Vasili Kuragin, a man of high\nrank and importance, who was the first to arrive at her reception.',
 'Anna\nPavlovna had had a cough for some days.',
 'She was, as she said, suffering\nfrom la grippe; grippe being then a new word in St. Petersburg, used\nonly by the elite.',
 'All her invitations without exception, written in

As you can see, there are many unwanted characters in each sentence: arbitrary newlines (\n), double hyphens (--) etc.

For the task at hand (learning fasttext embeddings), we might want to preprocess the text i.e. get rid of punctuation and redundant symbols, lowercase each word. We could also apply stemming or lemmatization, but fasttext gives us the freedom not to do so, preserving some morphology-related information.

In [10]:
def clean_wap_sentence(sentence: str) -> str:
    """
    Cleans a sentence by performing the following operations:
    1. Replaces newline characters (`\n`) with a space.
    2. Replaces double hyphens (`--`) with a space.
    3. Removes all punctuation except for apostrophes.
    4. Converts the sentence to lowercase.

    Args:
        sentence (str): The input sentence to be cleaned.

    Returns:
        str: The cleaned sentence.
    """
    sentence = sentence.replace("\n", " ").replace("--", " ")
    sentence = re.sub(r"[^\w\s']", "", sentence)
    sentence = sentence.lower().strip()

    return sentence


In [11]:
# Preprocess each sentence
processed_sentences = []

for sentence in sentences:
    processed_sentences.append(clean_wap_sentence(sentence))

# Join processed sentences with newlines to match expected gensim format
final_text = "\n".join(processed_sentences)

with open(wap_cleaned_file_path, 'w') as outfile:
    outfile.write(final_text)

# Keep sentence count for gensim model parameter setting
wap_sentences_count = len(processed_sentences)

In [12]:
# Take a look at what we obtained so far
!sed -n 1,4p war_and_peace_cleaned.txt

well prince so genoa and lucca are now just family estates of the buonapartes
but i warn you if you don't tell me that this means war if you still try to defend the infamies and horrors perpetrated by that antichrist i really believe he is antichrist i will have nothing more to do with you and you are no longer my friend no longer my 'faithful slave' as you call yourself
but how do you do
i see i have frightened you sit down and tell me all the news


Now that we have the preprocessed data, let's train our model at last!

For this task we will use [Gensim's implementation](https://radimrehurek.com/gensim/models/fasttext.html) of Fasttext. It has a little overhead as compared to the [vanilla implementation](https://fasttext.cc/), but a really nice API which also supports finetuning existing models.


**Please read carefully these notes in order to pass the tests:**
0. Set the learning rate to 3e-2. Gensim's fasttextwill linearly decrease this value as epochs go on.
1. You are free to experiment with the vector size,
but know that it will drastically affect the performance.
Values below 64 may result in embeddings to struggle to correctly represent the words.
Values above 300 will slow you down and will likely not offer any compensation for that.
We recommend a value of 100 to safely pass the test as a happy mean
2. Be sure to use skip-gram mode. It usually gives a better semantic representation of the learned vectors at the expense of a relatively slow training.
3. 5 epochs is likely enough for the amount of data you have, but feel free to experiment.
4. Remember to set the seed, even though it will likely not make you model fully determenistic if you use more than one thread.
5. Window size is an important parameter. The wider the window, the broader sentence-level context a given embedding captures at the expense of own semantic uniqueness. Feel free to experiment. The window size of 5 will likely be a good starting point to safely pass the assert wall.

Note that we do not use any train\test split. You, the human, is the ultimate referee for this task. If you feel that that the embeddings make sense (and they pass the test), then so be it.

In [13]:
# Use PathLineSentence to stream data from the file
wap_sentences = PathLineSentences(wap_cleaned_file_path)

# Train the FastText model using Gensim
model = FastText(
    sentences=wap_sentences,
    vector_size=100,
    window=5,
    min_count=5,
    sg=1,
    seed=seed,
    epochs=5,
    alpha=3e-2
)

model.save("wap_fasttext_model.bin")

You can play around with the model you trained below.

In [15]:
print(*model.wv.most_similar('peace', topn=10), sep='\n')

('hope', 0.8608812689781189)
('religion', 0.8507006764411926)
('secrecy', 0.850305438041687)
('source', 0.8461160063743591)
('secret', 0.8434459567070007)
('success', 0.8384630084037781)
('decision', 0.8357839584350586)
('chance', 0.8334247469902039)
('management', 0.8301828503608704)
('disgrace', 0.8294393420219421)


Fasttext allows us to reconstruct out-of-vocabulary words using word n-grams...

In [16]:
'computation' in model.wv.key_to_index

False

...even though they may be too far from the learned context.

In [18]:
print(*model.wv.most_similar('computation', topn=10), sep='\n')

('consultation', 0.9688815474510193)
('gravitation', 0.9675397872924805)
('subordination', 0.96674644947052)
('deputation', 0.9659308195114136)
('population', 0.9644306302070618)
('combination', 0.9631946682929993)
('participation', 0.9623448848724365)
('manifestation', 0.9616678953170776)
('civilization', 0.9614068865776062)
('proclamation', 0.9590864777565002)


In [19]:
# Degugging area
model_vocabulary = set(model.wv.key_to_index.keys())
most_similar_to_peace = list(zip(*model.wv.most_similar('peace', topn=50)))[0]

assert model.vector_size >= 64 and model.vector_size <= 300, 'Please check your embedding size.'
assert model.sg == 1, 'Please use skip-gram method for consistency. Also, despite being faster, CBOW usually generates inferior word embeddings for rare words.'
assert model.alpha == 3e-2, 'It is expected that you overwrite the default alpha for this task.'
assert len(model_vocabulary) > 5500 and len(model_vocabulary) < 7500, 'There is something wrong with your tokenization. Check the pipeline and use a default *ngram=1*'
assert 'religion' in most_similar_to_peace, 'Embeddings look odd. Make sure to follow instructions!'

print('Congrats!')

Congrats!


# Finetune using quora dataset

Now that we have embeddings representing the semantic space as was perceived by !Leo Tolstoy, let's move on to more up-to-date representations.

One problem is that our model's vocabulary lacks knowledge about modern day words. Even though we can reconstruct said words using fasttext n-grams, they lack contextual meaning.
Another problem is that the context in which certain words are used could have changed throughout the years.

Hence, our task would be not to overwrite the learned vectors, but to enrich them with some fresh data.

In [20]:
# download the data
!wget https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1 -O ./quora_raw.txt
# alternative download link: https://yadi.sk/i/BPQrUu1NaTduEw

--2025-10-09 19:44:17--  https://www.dropbox.com/s/obaitrix9jyu84r/quora.txt?dl=1
Resolving www.dropbox.com (www.dropbox.com)... 162.125.5.18, 2620:100:601c:18::a27d:612
Connecting to www.dropbox.com (www.dropbox.com)|162.125.5.18|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1 [following]
--2025-10-09 19:44:17--  https://www.dropbox.com/scl/fi/p0t2dw6oqs6oxpd6zz534/quora.txt?rlkey=bjupppwua4zmd4elz8octecy9&dl=1
Reusing existing connection to www.dropbox.com:443.
HTTP request sent, awaiting response... 302 Found
Location: https://uc53cba8fcabb9d7a40edbbfc570.dl.dropboxusercontent.com/cd/0/inline/Cy4zeG2cx9PXcfWIL-ntgCdyAdC4-eK-FubHmVlwzWarQKNhEWNAtvcjN_OlnbCy8OENF9eBQPNqsCsmLxvBdCwnvE1XzNWTOMUA6-QJMl3BJEQwlgOFiZ4Ur4cPzsAXl7E/file?dl=1# [following]
--2025-10-09 19:44:18--  https://uc53cba8fcabb9d7a40edbbfc570.dl.dropboxusercontent.com/cd/0/inline/Cy4zeG2cx9P

In [21]:
# Glimpse at the quora dataset
!sed -n 1,5p quora_raw.txt

Can I get back with my ex even though she is pregnant with another guy's baby?
What are some ways to overcome a fast food addiction?
Who were the great Chinese soldiers and leaders who fought in WW2?
What are ZIP codes in the Bay Area?
Why was George RR Martin critical of JK Rowling after losing the Hugo award?


As before, we need to standardize the data for training.

Luckily for us, each line already consists of a complete sentence (even better, in ASCII!).

In [22]:
# Define a function to preprocess a single line
def preprocess_line(line: str) -> str:
    """
    Preprocesses a single line of text by:
    1. Removing all punctuation except apostrophes.
    2. Converting the text to lowercase.

    Args:
        line (str): The input line of text to preprocess.

    Returns:
        str: The preprocessed line of text.
    """

    line = re.sub(r"[^\w\s']", "", line)
    line = line.lower().strip()

    return line


# Preprocess the text file line by line
def preprocess_file(input_file_path: str, output_file_path: str) -> int:
    """
    Reads a text file line by line, preprocesses each line,
    and writes the preprocessed lines to a new file.

    Args:
        input_file_path (str): Path to the input text file.
        output_file_path (str): Path to the output text file where preprocessed lines will be saved.

    Returns:
        int: The total number of lines processed in the input file.
    """

    lines_processed = 0
    with open(input_file_path, "r", encoding="utf-8") as infile, open(output_file_path, "w", encoding="utf-8") as outfile:
        for line in infile:
            line = preprocess_line(line.strip())
            if not line:
                continue
            outfile.write(line + "\n")
            lines_processed += 1

    return lines_processed


# Paths to the input and output files
quora_raw_file_path = 'quora_raw.txt'
quora_processed_file_path = 'quora_processed.txt'

# Preprocess the file and count the number of lines processed
quora_sentences_count = preprocess_file(quora_raw_file_path, quora_processed_file_path)

print(f'Lines processed: {quora_sentences_count}')


Lines processed: 537272


The catch here is that we continue trainining the model using a larger dataset with a pretty different thematics, so in order not to completely overwrite existing War and Peace "vibe" it may be a good idea to proceed at where we left off previously (learning rate-wise) and/or not to be too crazy about the number of epochs.  

In [32]:
# As before, use gensim's tools to iterate over the lines
quora_sentences = PathLineSentences(quora_processed_file_path)

# Load the model in order to finetune it
finetuned_model = FastText.load("wap_fasttext_model.bin")

# Update existing vocabulary with the new sentences
finetuned_model.build_vocab(corpus_iterable=quora_sentences, update=True)

# Continue training the model on the new data
# Remember to correctly pass the parameters
finetuned_model.train(
    corpus_iterable=quora_sentences,
    total_examples=quora_sentences_count,
    epochs=5,
    start_alpha=finetuned_model.min_alpha
)



finetuned_model.save("wap_quora_fasttext_model.bin")

In [33]:
finetuned_model_vocabulary = set(finetuned_model.wv.key_to_index.keys())
new_words = finetuned_model_vocabulary - model_vocabulary

print("Number of new words added:", len(new_words))
print("New words:", list(new_words)[10:20])

Number of new words added: 23878
New words: ['addicts', 'alphabets', 'kettle', 'd3200', 'networking', 'idiom', 'ballast', 'severance', 'cat', 'bracelet']


In [34]:
def print_comparison(
    model1: Union[FastText, None],
    model2: Union[FastText, None],
    word: str = 'peace',
    top_n: int = 10
) -> None:
    """
    A utility function that compares the top-N most similar words for a given query word across two FastText models.
    Prints the comparison in a tabular format to visually assess changes after finetuning.

    Args:
        model1 (FastText | None): The first FastText model (original or pre-finetuning).
        model2 (FastText | None): The second FastText model (post-finetuning).
        word (str, optional): The word for which similar words are retrieved.
        top_n (int, optional): The number of top similar words to compare.

    Returns:
        None: Prints results directly to the console.
    """
    q0 = model1.wv.most_similar(word, topn=top_n)
    q1 = model2.wv.most_similar(word, topn=top_n)

    # Print the header
    print(f'Query: {word}\n')
    print(f"{'pos':<5} {'model_1':<15} {'score_1':<10} {'model_2':<15} {'score_2':<10}")
    print("-" * (5 + 15 + 10 + 15 + 10))

    # Print each entry up to top_n
    for pos, (word0, score0), (word1, score1) in zip(range(top_n), q0, q1):
        # Format the scores to 4 decimal places
        print(f"{pos:<5} {word0:<15} {score0:.4f}     {word1:<15} {score1:.4f}")


In [35]:
# Call the function to display results
print_comparison(model, finetuned_model, word='war', top_n=10)

Query: war

pos   model_1         score_1    model_2         score_2   
-------------------------------------------------------
0     inevitability   0.8353     warg            0.9371
1     probability     0.8333     ministry        0.9306
2     nobility        0.8243     codevita        0.9240
3     dissolution     0.8177     inevitability   0.9228
4     warfare         0.8170     actuarial       0.9228
5     campaigns       0.8164     abolition       0.9219
6     vital           0.8149     revolution      0.9203
7     campaign        0.8129     microevolution  0.9193
8     universal       0.8114     dissolution     0.9192
9     revolution      0.8094     vita            0.9190


In [36]:
# The usual assert wall for debugging and testing
assert finetuned_model.alpha <= model.min_alpha, 'setting a learning rate too high will result in model forgetting previous information, especially when finetuning on a larger dataset!'
assert len(new_words) > 20_000 and len(new_words) < 30_000, 'There is likely something wrong with your preprocessing pipeline. There must be more new words after finetuning!'
assert 'religion' in list(zip(*finetuned_model.wv.most_similar('peace', topn=50)))[0], 'Embeddings changed dramatically! Consider setting lower start_apha and fewer epochs.'
assert 'evolution' in list(zip(*finetuned_model.wv.most_similar('war', topn=50)))[0], 'Embeddings didn\'t change as was expected! Consider setting higher start_apha and more epochs, check *total_examples* param.'

print('Gratz!')

Gratz!


In practice, you may want to experiment with different fine-tuning strategies e.g. varying the alpha parameter, training for more epochs, or even trying to "freeze" some embeddings while updating only certain parts of the vocabulary.

# Implement a way to search similar quora questions given a new query

## Straightforward numpy array-based search

Next, if we are to do a semantic search engine, it is required to be able to match a new sentence to a single vector. The easiest way to do so is to average individual word-level embeddings.

Alternatively, you may try out pooling (min, max, mean, softmax) strategies and [TF-IDF](https://www.geeksforgeeks.org/understanding-tf-idf-term-frequency-inverse-document-frequency/) . TF-IDF approach requires to precalculate IDF scores across existing corpus and multiply by a TF for a given word in a given sentence.

Also, it is expected that you will use cosine similarity as a vector distance metric later, so it might be a good idea to normalize all the vectors before putting them into a database since it will effectively turn a cosine similarity calculation into taking a dot product, which will speed the things up:
$$
cos(\phi) = \frac{<u, v>}{|u||v|}.
$$

In [48]:
def normalize_vector(embedding: np.ndarray) -> np.ndarray:
    """
    Normalizes a given vector to have unit length.

    Args:
        embedding (np.ndarray): A NumPy array representing the vector to normalize.

    Returns:
        np.ndarray: A normalized vector with unit length.
    """

    norm = np.linalg.norm(embedding)
    if norm == 0:
        return embedding
    embedding = embedding / norm
    return embedding


def get_text_embedding(text: str, model: FastText) -> np.ndarray:
    """
    Computes the embedding for a given text using a pre-trained FastText model.
    The embedding is the mean of word embeddings for words in the text, normalized to unit length.

    Args:
        text (str): The input text to embed.
        model (FastText): The FastText model to use for generating embeddings.

    Returns:
        np.ndarray: The normalized embedding for the input text.
                    If no valid words are found in the text, returns a zero vector.
    """

    words = preprocess_line(text).split()
    valid_embeddings = [model.wv[word] for word in words if word in model.wv.key_to_index]
    if not valid_embeddings:
        return np.zeros(model.vector_size, dtype=np.float32)
    mean_emb = np.mean(valid_embeddings, axis=0)
    normalized_word_embedding = normalize_vector(mean_emb)

    return normalized_word_embedding


In [49]:
# Debugging and testing area
question = "Love peace"
text_embedding = get_text_embedding(question, finetuned_model)
dummy_word_embedding = (finetuned_model.wv['love'] + finetuned_model.wv['peace']) / 2

assert text_embedding.shape[0] == finetuned_model.wv.vector_size, "Check axis along which values are aggregated"
assert all(normalize_vector(dummy_word_embedding) == text_embedding), "You need to return a normalized mean embedding for now, nothing fancy!"

print('Good job!')

Good job!


Finally, let's implement a similar sentence search.

Your task will be to match each quora question to an embedding and store these vectors in a way easy enough to conduct a search operation.

In order this to look like a search engine, new user input should also be converted to an embedding in order to perform, well, a search in our existing embedding storage.

First, let's do it in a straightforward way: create a numpy array where existing quora questions embeddings will be stored.

Be sure to apply the same preprocessing we used during the model training!

In [50]:
def create_embeddings_storage(quora_file: str, model: FastText) -> np.ndarray:
    """
    Creates a NumPy array of normalized embeddings for all questions in a dataset.

    Each line in the input file represents a question, which is preprocessed using
    'preprocess_line' function and embedded using 'get_text_embedding' function.

    Args:
        quora_file (str): Path to the file containing the preprocessed questions, one per line.
        model (FastText): The FastText model used to generate embeddings.

    Returns:
        np.ndarray: A 2D NumPy array where each row corresponds to the normalized embedding
                    of a question in the dataset. Shape: (num_questions, embedding_dim).
    """
    embeddings_list: List[np.ndarray] = []

    with open(quora_file, "r", encoding="utf-8") as f:
        for line in f:
            emb = get_text_embedding(line, model)
            embeddings_list.append(emb)

    # Convert the list of embeddings to a NumPy array
    return np.array(embeddings_list)

In [51]:
# Create a storage array for normalized embeddings
embeddings_storage_np = create_embeddings_storage("quora_processed.txt", finetuned_model)

In [52]:
with open(quora_processed_file_path, 'r') as f:
    for line in f:
        test_embedding = get_text_embedding(line, finetuned_model)
        break

assert embeddings_storage_np.shape == (quora_sentences_count, finetuned_model.vector_size), "The vector storage must be of size (num_quora_sentences, embedding_dim)"
assert embeddings_storage_np[0].mean() == test_embedding.mean(), 'Embedding of the first quora question does not correspond to the first enty in the storage!'
print('Looking good so far!')

Looking good so far!


In [53]:
def find_closest_match_np(query: str, model, embeddings_storage: np.ndarray, k: int = 1) -> Tuple[List[int], List[float]]:
    """
    Finds the closest match(es) to a new question in the database using cosine similarity.

    This function preprocesses the input question, calculates its embedding, and then computes
    cosine similarities between the new question's embedding and a pre-existing database of embeddings.
    It returns the indices of the top-k most similar questions along with their similarity scores.

    Parameters:
        query (str): The input question to be compared.
        model: The trained FastText model used to generate word embeddings.
        embeddings_storage (np.ndarray): A numpy array containing the precomputed and normalized embeddings of all questions in the database.
        k (int, optional): The number of closest matches to return. Defaults to 1.

    Returns:
        Tuple[List[int], List[float]]:
            - List[int]: A list of indices of the top-k most similar questions in the database.
            - List[float]: A list of similarity scores corresponding to these top-k matches.
    """

    # Preprocess the query and embed the question
    query_embedding = get_text_embedding(query, model)

    # Compute cosine similarity
    # Dot product if the vectors are normalized embeddings
    # You might want to look up np.dot
    similarities = np.dot(embeddings_storage, query_embedding)

    # Get the indices of the top-k most similar questions
    # You might want to look up np.argsort
    top_k_indices = np.argsort(similarities)[-k:][::-1]
    top_k_similarities = similarities[top_k_indices]
    top_k_indices = top_k_indices.tolist()
    top_k_similarities = top_k_similarities.tolist()

    return top_k_indices, top_k_similarities


For now, we will store all the quora questions in RAM using a python list.

In practice, you would consider a separate database with a retrieval by index done in constant time.

In [54]:
# Store original questions for visual testing
quora_questions_list = []
with open(quora_raw_file_path, 'r') as file:
    for line in file:
        quora_questions_list.append(line.strip())

print(f"Total questions processed: {len(quora_questions_list)}")

Total questions processed: 537272


In [55]:
def fetch_and_display_closest_match(query_function: Callable[..., Tuple[List[int], List[float]]], **kwargs) -> Tuple[List[int], List[float]]:
    """
    A utility function to fetch and display the closest matching questions
    from the Quora dataset for a given query.

    This function takes a query, uses the provided query function to retrieve
    the top-k most similar questions, and prints them along with their similarity scores.

    Args:
        query_function (Callable): A function that takes query-related parameters and returns
                                    the indices of the top-k most similar questions and their similarity scores.
                                    It should return a tuple (List[int], List[float]).
        **kwargs: Additional arguments to be passed to the query_function, including the query text.

    Returns:
        Tuple[List[int], List[float]]: A tuple containing:
            - A list of indices corresponding to the top-k most similar questions in the dataset.
            - A list of similarity scores for each of these questions.
    """

    # Retrieve the top-k indices and their similarity scores from the query function
    top_k_indices, top_k_similarities = query_function(**kwargs)

    # Fetch the actual questions based on the retrieved indices
    top_k_questions = [quora_questions_list[i] for i in top_k_indices]

    # Print the query and the top-k results with their similarity scores
    print(f"Query: {kwargs['query']}")
    print("\nTop Matches:")
    for i, (question, similarity) in enumerate(zip(top_k_questions, top_k_similarities), 1):
        print(f"{i}. {question} (Similarity: {similarity:.4f})")

    return top_k_indices, top_k_similarities


In [56]:
# testing area
new_question = "How can I find inner peace?"

top_k_indices, top_k_similarities = fetch_and_display_closest_match(query_function=find_closest_match_np,
                                                                    query=new_question, model=finetuned_model,
                                                                    embeddings_storage=embeddings_storage_np,
                                                                    k=10)

assert 446084 in top_k_indices, 'Your embeddings look odd. = ('

Query: How can I find inner peace?

Top Matches:
1. How can I create inner peace? (Similarity: 0.9944)
2. How can I find passion? (Similarity: 0.9943)
3. How can I find a career mentor? (Similarity: 0.9928)
4. How can I find inspiration? (Similarity: 0.9921)
5. How can I find a programmer mentor? (Similarity: 0.9918)
6. How can I find a successful mentor? (Similarity: 0.9917)
7. How can I find a startup mentor? (Similarity: 0.9914)
8. How can I prevent business failure? (Similarity: 0.9911)
9. How can I find a job? (Similarity: 0.9910)
10. How can I find app developers? (Similarity: 0.9908)


Please, feel free to play with your new search engine!

In [57]:
new_question = "What is the future of artificial intelligence?"

_, _ = fetch_and_display_closest_match(query_function=find_closest_match_np,
                                       query=new_question, model=finetuned_model,
                                       embeddings_storage=embeddings_storage_np,
                                       k=10)


Query: What is the future of artificial intelligence?

Top Matches:
1. What is the future of artificial intelligence? (Similarity: 1.0000)
2. What is the scope of Artificial Intelligence? (Similarity: 0.9982)
3. What is the future of OES Technical consultant? (Similarity: 0.9976)
4. What is the future of Chinese economy? (Similarity: 0.9975)
5. What is the future of Pharmaceutical industry? (Similarity: 0.9974)
6. What is the relative density of oil? (Similarity: 0.9973)
7. What is the future of internet piracy? (Similarity: 0.9973)
8. What is the future of electrical grid? (Similarity: 0.9973)
9. What is the future of Social Media? (Similarity: 0.9970)
10. What is the future of Behavioural Economics? (Similarity: 0.9970)


In [58]:
import time

iterations = 100

start_time = time.time()

for _ in range(iterations):
    _, _ = find_closest_match_np('hello there', finetuned_model, embeddings_storage_np, k=10)

time_elapsed = time.time() - start_time
average_time = time_elapsed / iterations

print(f'Time elapsed: {time_elapsed:.4f} seconds.')
print(f'Average time: {average_time:.4f} seconds.')

Time elapsed: 12.8304 seconds.
Average time: 0.1283 seconds.


Please note that in practice frequent queries will be cached.

# Vector database search

However, the main concern is that often your data will be too large to fit into RAM, let alone VRAM, so an option better than numpy array search is needed. Also, we can not afford to spend O(n) search time for each request.

Vector databases i.e. databases specifically designed to store vectors and conduct fast retrieval operations solve the speed problem at the expense of having to keep the storage in RAM. In practice, a collections of machines (shards) may host different chunks of data in an interconnected way.

Below is an already implemented way of building an embedding storage. You may want to look it through carefully.

You can read more about [FAISS](https://faiss.ai/index.html) library and [HNSW](https://arxiv.org/abs/1603.09320), which is, in short, a k-nearest neighbors search on steroids.

In [59]:
def build_faiss_hnsw_index(dimension: int, ef_construction: int = 200, M: int = 32) -> faiss.IndexHNSWFlat:
    """
    Builds a FAISS HNSW index for cosine similarity.

    This function initializes a HNSW (Hierarchical Navigable Small World) index for efficient approximate nearest neighbor search
    based on cosine similarity, using the FastText model's normalized embeddings.

    Parameters:
        dimension (int): Dimensionality of the embeddings (size of each vector).
        ef_construction (int, optional): Trade-off parameter between index construction speed and accuracy. Default is 200.
        M (int, optional): Number of neighbors in the graph, controlling the memory and accuracy tradeoff. Default is 32.

    Returns:
        index (faiss.IndexHNSWFlat): Initialized FAISS HNSW index.
    """
    index = faiss.IndexHNSWFlat(dimension, M)  # HNSW index
    index.hnsw.efConstruction = ef_construction  # Construction accuracy
    index.metric_type = faiss.METRIC_INNER_PRODUCT  # Cosine similarity via normalized vectors
    return index


def populate_faiss_index(index: faiss.Index, model, dataset_path: str, batch_size: int = 10000):
    """
    Populates the FAISS HNSW index with normalized embeddings from the dataset.

    This function reads the dataset line by line, preprocesses each question, computes its embedding,
    and adds the embeddings to the FAISS index in batches.

    Parameters:
        index (faiss.Index): FAISS index to populate.
        model: Trained FastText model used to generate embeddings.
        dataset_path (str): Path to the dataset file (one question per line).
        batch_size (int, optional): Number of questions to process at a time. Default is 10000.
    """
    with open(dataset_path, "r") as f:
        buffer = []
        for line in f:
            # Preprocess the line and get its embedding
            question = preprocess_line(line.strip())
            embedding = get_text_embedding(question, model)
            buffer.append(embedding)

            # Add embeddings to the index in batches
            if len(buffer) >= batch_size:
                index.add(np.array(buffer, dtype=np.float32))
                buffer = []  # Clear the buffer after adding

        # Add remaining embeddings (if any) after loop finishes
        if buffer:
            index.add(np.array(buffer, dtype=np.float32))


def search_faiss_index(embeddings_storage: faiss.Index, query: str, model, k: int = 5) -> Tuple[List[int], List[float]]:
    """
    Searches the FAISS index for the closest matches to a query.

    This function computes the embedding for the input query, searches the FAISS index for the most similar questions,
    and returns the indices and distances (cosine similarity) of the top-k matches.

    Parameters:
        embeddings_storage (faiss.Index): FAISS index to search.
        query (str): The input query string to search for.
        model: Trained FastText model used to generate embeddings.
        k (int, optional): Number of closest matches to retrieve. Default is 5.

    Returns:
        Tuple[List[int], List[float]]:
            - List[int]: A list of indices of the top-k most similar questions.
            - List[float]: A list of distances (cosine similarity) of the top-k results.
    """
    # Preprocess and normalize the query embedding
    query_embedding = get_text_embedding(query, model)

    # Search the embeddings_storage (FAISS index)
    top_k_distances, top_k_indices = embeddings_storage.search(np.array([query_embedding], dtype=np.float32), k)

    # Match return format with that used in numpy storage search
    top_k_indices_list = top_k_indices[0].tolist()
    top_k_distances_list = top_k_distances[0].tolist()

    return top_k_indices_list, top_k_distances_list


Building a faiss may take some time, it's ok. In some sense, we trade computations during index constuction for query time later.

In [ ]:
# Define the dimensions of the embedding vectors
embedding_dimension = finetuned_model.vector_size  # Depends on the FastText model

# Build the HNSW index
hnsw_index = build_faiss_hnsw_index(embedding_dimension)

# Populate the index from the quora_processed.txt dataset
populate_faiss_index(hnsw_index, finetuned_model, "quora_processed.txt")


Note that we built a FAISS index around Euclidian distance, not cosine similarity, to be completely honest:

$$
cosine \space similarity = 1 - \frac{euclidian \space distance^2}{2}
$$

Note that the results obtained may not entirely repeat a numpy-based search since HNSW index is built upon approximate computations.

In [ ]:
# Example query
query_text = "How can I find inner peace?"

top_k_indices, top_k_similarities = fetch_and_display_closest_match(query_function=search_faiss_index,
                                                                    query=query_text, model=finetuned_model,
                                                                    embeddings_storage=hnsw_index,
                                                                    k=10)

In [ ]:
import time

iterations = 100

start_time = time.time()

for _ in range(iterations):
    _, _ = search_faiss_index(hnsw_index, query_text, finetuned_model, k=10)

time_elapsed = time.time() - start_time
average_time = time_elapsed / iterations

print(f'Time elapsed: {time_elapsed:.4f} seconds.')
print(f'Average time: {average_time:.4f} seconds.')

Note that we achieved a two orders of magnitude speed-up!